In [1]:
import captum
import torch
import utils
from model.data_loader import DataLoader
from torch import device as dev
from captum.attr import LayerIntegratedGradients, LayerGradientShap, LayerFeatureAblation, TokenReferenceBase, visualization, Saliency
from captum_viz import interpret_sequence
from captum_viz import *
import model.haard_net as net
import utils


In [2]:
%load_ext autoreload

Parameter Setup

In [3]:
def _subseq(sequence, weights, top_std, predicted_label, true_label, before_after, sampleIdx):
    correct = predicted_label == true_label
    # print("predicted", predicted_label, "| true", true_label, "|", correct)

    # print(weights.shape)
    # print(weights[:10])
    mean, std = weights.mean(), weights.std()
    indices = np.where(weights > (mean + top_std*std))[0]

    allseqs = []
    cols = ['left', 'right', 'subsequence', 'predicted', 'true', 'classification', 'inputSequence']
    for idx in indices:
        left = idx - before_after
        if left < 0: left = 0
        right = idx + before_after + 1
        if right > len(sequence): right = len(sequence)
        
        subseq = ''.join(sequence[left:right])
        allseqs.append([left, right, subseq, predicted_label, true_label, correct, sampleIdx])
    # print(pd.DataFrame(allseqs, columns=cols))
    newSeqs = []
    i = 0
    while i < len(allseqs):
        row = allseqs[i]
        newSeqRow = row
        currleft = row[0]
        currright = row[1]
        j = i + 1
        if j < len(allseqs):
            nextLeft = allseqs[j][0]
        else:
            newSeqs.append(row)
            break
        while nextLeft < currright:
            currright = allseqs[j][1]
            j += 1
            if j >= len(allseqs): break
            nextLeft = allseqs[j][0]
        newRight = allseqs[j-1][1]
        newSeqRow[1] = newRight
        newSeqRow[2] = ''.join(sequence[currleft : newRight])
        newSeqs.append(newSeqRow)
        i = j

    allseqs = pd.DataFrame(newSeqs, columns=cols)
    # print(pd.DataFrame(allseqs, columns=cols))
    return allseqs

In [4]:
MODEL_DIR = 'experiments/attention2_2/'
DATA_DIR = 'data/'
sentences_file = DATA_DIR + 'test/sentences.txt'
labels_file = DATA_DIR + 'test/labels.txt'
weights = MODEL_DIR + 'best.pth'
params = utils.Params(MODEL_DIR+'params.json')
params.batch_size = 1
params.vocab_size = 25
params.number_of_classes = 10
params.cuda = torch.cuda.is_available()

Data setup

In [5]:
loader = DataLoader(DATA_DIR, params)
data = loader.load_data(['train', 'val', 'test'], DATA_DIR)
test_data = data['test']
test_data_iterator = loader.data_iterator(test_data, params, shuffle=False)

# test_batch, label_batch = next(test_data_iterator)

classes = ['Extracellular', 'Plastid', 'Cytoplasm', 'Mitochondrion', 
'Nucleus', 'ER', 'Golgi', 'Membrane', 'Lysosome', 'Peroxisome']

Model setup

In [6]:
model = net.Net(params).cuda() if params.cuda else net.Net(params)
utils.load_checkpoint(weights, model)
# checkpoint = torch.load(weights, map_location=dev('cpu'))
# model.load_state_dict(checkpoint['state_dict'])
model.eval()

Loading: experiments/attention2_2/best.pth
Model summary
Net(
  (embedding): Embedding(25, 20)
  (lstm): LSTM(20, 40, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=80, out_features=10, bias=True)
  (w_omega): Linear(in_features=80, out_features=512, bias=False)
  (u_omega): Linear(in_features=512, out_features=1, bias=False)
)


Net(
  (embedding): Embedding(25, 20)
  (lstm): LSTM(20, 40, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=80, out_features=10, bias=True)
  (w_omega): Linear(in_features=80, out_features=512, bias=False)
  (u_omega): Linear(in_features=512, out_features=1, bias=False)
)

In [7]:
bool(model.use_attention)

False

**Visualizing Saliency**

In [8]:
layer_sal = Saliency(model)

In [9]:
num_steps = (params.test_size + 1) // params.batch_size
ig_vis_data_records = []
interpret_sequence_copy(model, loader, test_data_iterator, layer_sal, ig_vis_data_records, num_steps, verbose=True)

Step 0 / 2102


RuntimeError: One of the differentiated Tensors does not require grad

**Visualizing Integrated Gradient**

In [8]:
# sentences = open(sentences_file).read().splitlines()
# labels = open(labels_file).read().splitlines()
# token_reference = TokenReferenceBase(reference_token_idx=loader.pad_ind)
layer_ig = LayerIntegratedGradients(model, model.embedding)
ig_vis_data_records = []
ig_attn_vis_data_records = []

In [14]:
import pandas as pd
import numpy as np

In [9]:
num_steps = (params.test_size + 1) // params.batch_size
interpret_sequence_copy(model, loader, test_data_iterator, layer_ig, ig_vis_data_records, num_steps)

2102 [46:13<03:59,  1.52s/it]pred:  Cytoplasm ( 0.68 ) , delta:  0.00011937103958678877 , true: Membrane
100%|█████████▉| 2101/2102 [49:56<00:01,  1.15s/it]pred:  Cytoplasm ( 0.69 ) , delta:  0.007413580483083021 , true: Nucleus


ValueError: max() arg is an empty sequence

In [16]:
outseq = pd.DataFrame()
before_after = 2
top_std = 2 # look at top 2.25% sequences
for i, vd in enumerate(ig_vis_data_records):
    sequence = vd.raw_input
    weights = vd.word_attributions
    predicted_label = vd.pred_class
    true_label = vd.true_class
    sampleIdx = i
    out = _subseq(sequence, weights, top_std, predicted_label, true_label, before_after, sampleIdx)
    print(out)
    outseq = outseq.append(out)
    # if i > 
seqdir = 'subsequences'
if not os.path.exists(seqdir): os.makedirs(seqdir)
fname = f'above_top{top_std}std_subseqs_testData_beforeAfter{before_after}.csv'
fname = os.path.join(seqdir, fname)
outseq.to_csv(fname, index=False)

 Membrane           False   

   inputSequence  
0           2003  
1           2003  
   left  right   subsequence  predicted           true  classification  \
0     0     12  MSIMPVKNALAQ  Cytoplasm  Mitochondrion           False   
1    21     26         PAVKF  Cytoplasm  Mitochondrion           False   
2    53     58         RSSGE  Cytoplasm  Mitochondrion           False   
3    93     99        LVSRND  Cytoplasm  Mitochondrion           False   
4   109    115        KGRRTR  Cytoplasm  Mitochondrion           False   
5   130    135         TLPLN  Cytoplasm  Mitochondrion           False   

   inputSequence  
0           2004  
1           2004  
2           2004  
3           2004  
4           2004  
5           2004  
   left  right subsequence  predicted           true  classification  \
0     0      5       MSIMK  Cytoplasm  Mitochondrion           False   

   inputSequence  
0           2005  
   left  right subsequence      predicted     true  classification  \
0     0 

In [8]:
visualization.visualize_text(ig_vis_data_records)

In [9]:
params.attn = 1
attn_model = net.Net(params).cuda() if params.cuda else net.Net(params)
checkpoint = torch.load(weights, map_location=dev('cpu'))
attn_model.load_state_dict(checkpoint['state_dict'])
attn_model.eval()

Net(
  (embedding): Embedding(25, 20)
  (lstm): LSTM(20, 40, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (attention): Attention(
    (linear_in): Linear(in_features=512, out_features=512, bias=False)
    (linear_out): Linear(in_features=1024, out_features=512, bias=False)
    (softmax): Softmax(dim=-1)
    (tanh): Tanh()
  )
  (fc): Linear(in_features=80, out_features=10, bias=True)
)

In [10]:
bool(attn_model.use_attention)

True

In [11]:
layer_ig_attn = LayerIntegratedGradients(attn_model, attn_model.embedding)
interpret_sequence(attn_model, sentences[:5], train_batch, layer_ig_attn, ig_attn_vis_data_records)

pred:  Extracellular ( 0.99 ) , delta:  8.579375077033546e-08
pred:  Plastid ( 1.00 ) , delta:  8.487881277829956e-07
pred:  Cytoplasm ( 0.88 ) , delta:  3.6450237450580403e-07
pred:  Mitochondrion ( 1.00 ) , delta:  2.3572954965800363e-07
pred:  Cytoplasm ( 0.82 ) , delta:  7.547191882562032e-08


In [12]:
visualization.visualize_text(ig_attn_vis_data_records)

**GradientSHAP Visualization**

In [49]:
%autoreload 2

In [50]:
layer_shap = LayerGradientShap(model, model.embedding)
layer_shap_attn = LayerGradientShap(attn_model, attn_model.embedding)
shap_vis_data_records = []
shap_attn_vis_data_records = []

In [51]:
interpret_sequence(model, sentences[:5], train_batch.long(), layer_shap, shap_vis_data_records)

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)

In [ ]:
visualization.visualize_text(shap_vis_data_records)

In [ ]:
interpret_sequence(attn_model, sentences[:5], train_batch, layer_shap_attn, shap_attn_vis_data_records)

In [ ]:
visualization.visualize_text(shap_vis_attn_data_records)

**Feature Ablation Visualization**

In [41]:
model

Net(
  (embedding): Embedding(25, 20)
  (lstm): LSTM(20, 40, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (attention): Attention(
    (linear_in): Linear(in_features=512, out_features=512, bias=False)
    (linear_out): Linear(in_features=1024, out_features=512, bias=False)
    (softmax): Softmax(dim=-1)
    (tanh): Tanh()
  )
  (fc): Linear(in_features=80, out_features=10, bias=True)
)

In [52]:
layer_fa = LayerFeatureAblation(model, model.embedding)
fa_vis_data_records = []

In [55]:
interpret_sequence(model, sentences[:5], model.embedding(train_batch).long(), layer_fa, fa_vis_data_records)

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)

In [ ]:
layer_fa_attn = LayerFeatureAblation(model, attn_model.embedding)
fa_attn_vis_data_records = []

In [36]:
interpret_sequence(attn_model, sentences[:5], train_batch, layer_fa_attn, fa_attn_vis_data_records)

RuntimeError: The size of tensor a (673) must match the size of tensor b (20) at non-singleton dimension 3